In [591]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize

In [592]:
class FashionMNISTtask1(datasets.FashionMNIST):
  def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
    super(FashionMNISTtask1, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
    self.classes = self.classes[:6]

  def __getitem__(self, index):
    img, target = super(FashionMNISTtask1, self).__getitem__(index)
    if target < 6:
        return img, target
    else:
        return img, -1


In [593]:
class FashionMNISTtask2(datasets.FashionMNIST):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        super(FashionMNISTtask2, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
        self.classes = self.classes[6:]

    def __getitem__(self, index):
        img, target = super(FashionMNISTtask2, self).__getitem__(index)
        if target >= 6:
            return img, target
        else:
            return img, -1

In [594]:
transform = torchvision.transforms.Compose([ToTensor(), Normalize((0.5), (0.5))])

train_dataset_task1 = FashionMNISTtask1(root='./data1', train=True, transform=transform, download=True)
test_dataset_task1 = FashionMNISTtask1(root='./data1', train=False, transform=transform, download=True)

In [595]:
train_dataset_task2 = FashionMNISTtask2(root='./data2', train=True, transform=transform, download=True)
test_dataset_task2 = FashionMNISTtask2(root='./data2', train=False, transform=transform, download=True)

In [596]:
train_dataset_filtered_task1 = [data for data in train_dataset_task1 if data[1] != -1]
test_dataset_filtered_task1 = [data for data in test_dataset_task1 if data[1] != -1]

train_dataset_filtered_task2 = [data for data in train_dataset_task2 if data[1] != -1]
test_dataset_filtered_task2 = [data for data in test_dataset_task2 if data[1] != -1]

In [597]:
train_dataloader_task1 = DataLoader(train_dataset_filtered_task1, batch_size=64, shuffle=True)
test_dataloader_task1 = DataLoader(test_dataset_filtered_task1, batch_size=256, shuffle=False)

for X, y in train_dataloader_task1:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"SHape of y: {y.shape}, dtype: {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
SHape of y: torch.Size([64]), dtype: torch.int64


In [598]:
len(train_dataloader_task1)

563

In [599]:
train_dataloader_task2 = DataLoader(train_dataset_filtered_task2, batch_size=64, shuffle=True)
test_dataloader_task2 = DataLoader(test_dataset_filtered_task2, batch_size=256, shuffle=False)

In [600]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [601]:
def kaiming_normal_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')

In [602]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(28*28, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')
            elif isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        logits = self.classifier(x)

        return logits

In [603]:
class CNN(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32*14*14, hidden_size)
        self.relu3 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        logits = self.classifier(x)

        return logits

    def forward_features(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)

        return x


In [604]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [605]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}\n")


In [606]:
model_task1 = CNN(num_classes=6, hidden_size=512).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_task1.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [607]:
model_task1.classifier.out_features

6

In [608]:
epochs = 3
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader_task1, model_task1, loss_fn, optimizer)
  test(test_dataloader_task1, model_task1, loss_fn)
print("Done!")

torch.save(model_task1.state_dict(), "model_old.pth")

Epoch 1
---------------------------
Loss: 1.780460,    64/36000
Loss: 0.249530,  6464/36000
Loss: 0.308725, 12864/36000
Loss: 0.305284, 19264/36000
Loss: 0.165172, 25664/36000
Loss: 0.211944, 32064/36000
Test Error: 
 Accuracy: 92.7, Avg Loss: 0.200634

Epoch 2
---------------------------
Loss: 0.160993,    64/36000
Loss: 0.181258,  6464/36000
Loss: 0.105811, 12864/36000
Loss: 0.127520, 19264/36000
Loss: 0.214479, 25664/36000
Loss: 0.213127, 32064/36000
Test Error: 
 Accuracy: 92.8, Avg Loss: 0.198229

Epoch 3
---------------------------
Loss: 0.154834,    64/36000
Loss: 0.107329,  6464/36000
Loss: 0.234791, 12864/36000
Loss: 0.085211, 19264/36000
Loss: 0.053493, 25664/36000
Loss: 0.089896, 32064/36000
Test Error: 
 Accuracy: 93.4, Avg Loss: 0.194114

Done!


In [609]:
class ModifiedCNN(CNN):
    def __init__(self, num_classes, hidden_size, new_hidden_size):
        super(ModifiedCNN, self).__init__(num_classes=num_classes, hidden_size=hidden_size)
        self.new_fc = nn.Linear(hidden_size, new_hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.classifier = nn.Linear(new_hidden_size, num_classes)

    def forward(self, x):
        x = super().forward_features(x)
        x = self.new_fc(x)
        x = self.relu(x)
        x = self.classifier(x)
        return x

In [610]:
model = ModifiedCNN(num_classes=10, hidden_size=512, new_hidden_size=512)

In [611]:
model

ModifiedCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=6272, out_features=512, bias=True)
  (relu3): ReLU()
  (classifier): Linear(in_features=512, out_features=10, bias=True)
  (new_fc): Linear(in_features=512, out_features=512, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

# LWF

In [612]:
model_task2 = CNN(num_classes=6, hidden_size=512).to(device)
model_task1 = CNN(num_classes=6, hidden_size=512).to(device)

model_task2.load_state_dict(torch.load("model_old.pth"))
model_task1.load_state_dict(torch.load("model_old.pth"))

<All keys matched successfully>

In [613]:
model_task2.classifier.weight

Parameter containing:
tensor([[-0.0420, -0.0886,  0.0622,  ...,  0.0946, -0.0232, -0.0075],
        [-0.0529,  0.1091,  0.0897,  ..., -0.0259,  0.0137,  0.0191],
        [-0.0006, -0.0483,  0.0094,  ..., -0.0634,  0.0197,  0.0135],
        [-0.0516,  0.0060, -0.1399,  ..., -0.0296, -0.0193,  0.0175],
        [ 0.1343, -0.0259, -0.0238,  ...,  0.0130, -0.0305,  0.0570],
        [-0.1021,  0.0719, -0.0779,  ..., -0.0098,  0.0469,  0.0050]],
       requires_grad=True)

In [614]:
model_task2 = ModifiedCNN(num_classes=10, hidden_size=512, new_hidden_size=512)

In [615]:
model_task2.classifier.weight

Parameter containing:
tensor([[ 0.0296,  0.0304, -0.0359,  ..., -0.0186,  0.0120, -0.0285],
        [-0.0273, -0.0371,  0.0219,  ..., -0.0137,  0.0055,  0.0087],
        [-0.0288,  0.0023,  0.0200,  ...,  0.0004,  0.0245, -0.0188],
        ...,
        [ 0.0324,  0.0220, -0.0435,  ...,  0.0151, -0.0284,  0.0152],
        [-0.0039,  0.0393,  0.0344,  ...,  0.0249,  0.0198,  0.0044],
        [-0.0242, -0.0067, -0.0267,  ..., -0.0025, -0.0008, -0.0305]],
       requires_grad=True)

In [616]:
in_features = model_task1.classifier.in_features
out_features = model_task1.classifier.out_features

new_weights = torch.zeros_like(model_task2.classifier.weight)
new_biases = torch.zeros_like(model_task2.classifier.bias)

weight = model_task1.classifier.weight.data
bias = model_task1.classifier.bias.data

new_weights[:out_features, :] = weight
new_biases[:out_features] = bias

In [617]:
model_task2.classifier.weight = nn.Parameter(new_weights)
model_task2.classifier.bias = nn.Parameter(new_biases)

In [618]:
model_task2.classifier.weight

Parameter containing:
tensor([[-0.0420, -0.0886,  0.0622,  ...,  0.0946, -0.0232, -0.0075],
        [-0.0529,  0.1091,  0.0897,  ..., -0.0259,  0.0137,  0.0191],
        [-0.0006, -0.0483,  0.0094,  ..., -0.0634,  0.0197,  0.0135],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       requires_grad=True)

In [619]:
model_task2 = model_task2.to(device)
print("New head numbers: ", model_task2.classifier.out_features)

New head numbers:  10


In [620]:
for param in model_task1.parameters():
  param.requires_grad = False

In [621]:
# Initially freeze all the layers except for classifier
for name, param in model_task2.named_parameters():
    if name not in ['new_fc.weight', 'new_fc.bias']:
        param.requires_grad = False

In [622]:
for name, _ in model_task2.named_parameters():
    print(name)

conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
classifier.weight
classifier.bias
new_fc.weight
new_fc.bias


Changes in training and testing

In [623]:
def train(alpha, T):
    size = len(train_dataloader_task2.dataset)
    # We set net_new to evaluation mode to prevent it from being updated
    # while computing the distillation loss from the old model

    model_task2.train()
    for batch, (X, y) in enumerate(train_dataloader_task2):
        X, y = X.to(device), y.to(device)

        outputs = model_task2(X)
        soft_y = model_task1(X)

        loss1 = loss_fn(outputs, y)

        outputs_S = nn.functional.softmax(outputs[:, :out_features] / T, dim=1)
        outputs_T = nn.functional.softmax(soft_y[:, :out_features] / T, dim=1)

        loss2 = outputs_T.mul(-1 * torch.log(outputs_S))
        loss2 = loss2.sum(1)
        loss2 = loss2.mean() * T * T

        loss = loss1 + alpha * loss2

        loss.backward(retain_graph=True)
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")


In [624]:
def test(alpha, T):
  size = len(test_dataloader_task2.dataset)
  num_batches = len(test_dataloader_task2)
  model_task2.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in test_dataloader_task2:
      X, y = X.to(device), y.to(device)

      outputs = model_task2(X)
      soft_y = model_task1(X)

      loss1 = loss_fn(outputs, y)

      outputs_S = nn.functional.softmax(outputs[:, :out_features] / T, dim=1)
      outputs_T = nn.functional.softmax(soft_y[:, :out_features] / T, dim=1)

      loss2 = outputs_T.mul(-1 * torch.log(outputs_S))
      loss2 = loss2.sum(1)
      loss2 = loss2.mean() * T * T

      loss = loss1 * alpha + loss2 * (1 - alpha)

      test_loss += loss.item()
      _, predicted = outputs.max(1)
      correct += predicted.eq(y).sum().item()
      # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}")


In [625]:
def val(epoch):
    model_task2.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader_task1):
            X, y = X.to(device), y.to(device)
            outputs = model_task2(X)
            _, predicted_old = outputs.max(1)
            total += len(y)
            correct += predicted_old.eq(y).sum().item()
        print(f"Validation Acc: {100. * correct / total}\n")

In [626]:
T = 2
alpha = 0.9
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model_task2.parameters(), lr=0.0001, momentum=0.9, weight_decay=5e-4)

for epoch in range(5):
    print(f"Epoch {epoch+1}: ----------------------")
    if epoch > 0:
        for param in model_task2.parameters():
            param.requires_grad = True
        optimizer = torch.optim.SGD(model_task2.parameters(), lr=0.0001, momentum=0.9)
    train(alpha, T)
    test(alpha, T)
    val(epoch)

Epoch 1: ----------------------
Loss: 9.069983,    64/24000
Loss: 8.001513,  6464/24000
Loss: 7.533881, 12864/24000
Loss: 6.879340, 19264/24000
Test Error: 
 Accuracy: 0.1, Avg Loss: 2.552623
Validation Acc: 32.46666666666667

Epoch 2: ----------------------
Loss: 6.795416,    64/24000
Loss: 5.119720,  6464/24000
Loss: 4.309440, 12864/24000
Loss: 4.222644, 19264/24000
Test Error: 
 Accuracy: 87.8, Avg Loss: 1.057320
Validation Acc: 0.0

Epoch 3: ----------------------
Loss: 4.065773,    64/24000
Loss: 3.606124,  6464/24000
Loss: 3.684023, 12864/24000
Loss: 2.892900, 19264/24000
Test Error: 
 Accuracy: 92.2, Avg Loss: 0.699389
Validation Acc: 0.0

Epoch 4: ----------------------
Loss: 3.007957,    64/24000
Loss: 3.102300,  6464/24000
Loss: 3.289531, 12864/24000
Loss: 2.938645, 19264/24000
Test Error: 
 Accuracy: 93.2, Avg Loss: 0.559361
Validation Acc: 0.0

Epoch 5: ----------------------
Loss: 2.850488,    64/24000
Loss: 2.873625,  6464/24000
Loss: 2.997895, 12864/24000
Loss: 3.106426,